In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
from pprint import pprint
import torch
import sys
import yaml
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from Utils import TransformerSampler, ModelConfig, GenerationConfig
from model import Model
from statedict_mapping import convert_model_weights
torch.manual_seed(1337)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

Using device: cuda


In [13]:
sampler = TransformerSampler()
model = AutoModelForCausalLM.from_pretrained("/home/ubuntu/MechInter/1LayerModel/Models/TinyStories-1L-21M").to(device)
tokenizer = AutoTokenizer.from_pretrained("/home/ubuntu/MechInter/1LayerModel/Models/TinyStories-1L-21M")
tokenizer.pad_token = tokenizer.eos_token

with open('OneLM.yaml', 'r') as f:
    config_dict = yaml.safe_load(f)
    model_config = ModelConfig(config_dict)

modified_state_dict = convert_model_weights(model_config, model)
model = Model(model_config).to(device)
model.load_state_dict(modified_state_dict)

<All keys matched successfully>

In [15]:
prompts = [
    'Once upon a time',
    # 'In a galaxy far, far away',
    'The quick brown fox',
]

tokens = tokenizer(prompts, return_tensors='pt', padding=True, padding_side='left', truncation=True).to(device='cuda')
input_ids = tokens['input_ids']
attention_mask = tokens['attention_mask']

for _ in range(15):
    logits = model(input_ids = input_ids, attention_mask = attention_mask)

    # if running inference using transformers
    # logits = logits.logits
    
    next_token = sampler.decode(logits, temperature=0).to(device='cuda')
    print(tokenizer.batch_decode(next_token))
    input_ids = torch.cat([input_ids, next_token.unsqueeze(-1)], dim=1)
    attention_mask = torch.cat([attention_mask, torch.ones((attention_mask.shape[0], 1), dtype=attention_mask.dtype, device='cuda')], dim=-1)
# print(tokenizer.batch_decode(input_ids))


[' there', ' was']
[' were', ' happy']
[' two', ',']
[' friends', ' and']
[',', ' he']
[' Jack', ' and']
[' and', ' the']
[' Jill', ' bird']
['.', ' played']
[' They', ' together']
[' lived', ' at']
[' in', ' the']
[' a', ' park']
[' house', ' every']
[' with', ' day']
